In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import zipfile as zf
files = zf.ZipFile("formula1.zip", 'r')
files.extractall('directory to extract')
files.close()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import numpy as np # linear algebra
# data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import os
fnames = []
fpaths = []
for dirname, _, filenames in os.walk('/formula1data'):
    for filename in filenames:
        fnames.append(filename.split('.')[0])
        fpaths.append(os.path.join(dirname, filename))
        print(os.path.join(dirname, filename))

In [12]:

circuits = pd.read_csv('formula1data/circuits.csv', index_col=0, na_values=r'\N')
constructorResults = pd.read_csv('formula1data/constructor_results.csv', index_col=0, na_values=r'\N')
constructors = pd.read_csv('formula1data/constructors.csv', index_col=0, na_values=r'\N')
constructorStandings = pd.read_csv('formula1data/constructor_standings.csv', index_col=0, na_values=r'\N')
drivers = pd.read_csv('formula1data/drivers.csv', index_col=0, na_values=r'\N')
driverStandings = pd.read_csv('formula1data/driver_standings.csv', index_col=0, na_values=r'\N')
lapTimes = pd.read_csv('formula1data/lap_times.csv')
pitStops = pd.read_csv('formula1data/pit_stops.csv')
qualifying = pd.read_csv('formula1data/qualifying.csv', index_col=0, na_values=r'\N')
races = pd.read_csv('formula1data/races.csv', na_values=r'\N')
results = pd.read_csv('formula1data/results.csv', index_col=0, na_values=r'\N')
seasons = pd.read_csv('formula1data/seasons.csv', index_col=0, na_values=r'\N')
status = pd.read_csv('formula1data/status.csv', index_col=0, na_values=r'\N')

In [13]:
newResults = pd.merge(results,races,left_on='raceId',right_index=True,how='outer')

In [14]:
newResults.describe().T

,count,mean,std,min,25%,50%,75%,max
raceId,25863.0,5.317965e+02,2.996364e+02,0.00,293.000,515.000,7.850000e+02,1101.00
raceId_x,25840.0,5.314258e+02,2.994409e+02,1.00,293.000,514.000,7.840000e+02,1096.00
driverId,25840.0,2.617321e+02,2.686230e+02,1.00,56.000,163.000,3.600000e+02,856.00
constructorId,25840.0,4.862833e+01,5.973213e+01,1.00,6.000,25.000,5.800000e+01,214.00
number,25834.0,1.779008e+01,1.510484e+01,0.00,7.000,15.000,2.400000e+01,208.00
grid,25840.0,1.117906e+01,7.243725e+00,0.00,5.000,11.000,1.700000e+01,34.00
position,14989.0,7.942491e+00,4.806021e+00,1.00,4.000,7.000,1.100000e+01,33.00
positionOrder,25840.0,1.287601e+01,7.712391e+00,1.00,6.000,12.000,1.800000e+01,39.00
points,25840.0,1.877053e+00,4.169849e+00,0.00,0.000,0.000,2.000000e+00,50.00
laps,25840.0,4.597752e+01,2.980895e+01,0.00,22.000,52.000,6.600000e+01,200.00


In [15]:
columns = ["driverId", "fastestLap", "rank", "raceId_x", "racedId_y", "curcuitId","constructor", "position"]
newResults = newResults.reindex(columns=columns)
newResults.describe().T

,count,mean,std,min,25%,50%,75%,max
driverId,25840.0,261.732082,268.623016,1.0,56.0,163.0,360.0,856.0
fastestLap,7379.0,42.514162,16.835664,2.0,32.0,45.0,54.0,85.0
rank,7591.0,10.409959,6.162407,0.0,5.0,10.0,16.0,24.0
raceId_x,25840.0,531.425813,299.440908,1.0,293.0,514.0,784.0,1096.0
racedId_y,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
curcuitId,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
constructor,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
position,14989.0,7.942491,4.806021,1.0,4.0,7.0,11.0,33.0


In [16]:
newResults = newResults.loc[:,["driverId", "fastestLap", "rank", "raceId_x", "position"]]
#newResults.fillna(newResults.mean())

newResults.fillna(newResults.mean())

#------------------------------------------------
#----(Code 2) Selective Treatment----------------

for i in newResults.columns[newResults.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    newResults[i].fillna(newResults[i].mean(),inplace=True)

newResults.isna().sum()
newResults.describe().T

,count,mean,std,min,25%,50%,75%,max
driverId,25863.0,261.732082,268.503542,1.0,56.000000,163.000000,360.000000,856.0
fastestLap,25863.0,42.514162,8.992253,2.0,42.514162,42.514162,42.514162,85.0
rank,25863.0,10.409959,3.338414,0.0,10.409959,10.409959,10.409959,24.0
raceId_x,25863.0,531.425813,299.307726,1.0,293.000000,515.000000,783.000000,1096.0
position,25863.0,7.942491,3.658697,1.0,6.000000,7.942491,8.000000,33.0


In [17]:
#from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier

#imp = SimpleImputer(strategy="most_frequent")
#print(imp.fit_transform(newResults))
#newResults.isna().sum()
#newResults.describe().T


In [19]:
from matplotlib import rcParams
import warnings
from sklearn import preprocessing
from sklearn import utils

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

#newResults = SimpleImputer(missing_values=np.nan, strategy='mean')
#newResults = newResults.fit(X_train)
X = newResults.drop("position", axis=1)
y = newResults["position"]

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

warnings.filterwarnings("ignore")


# standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_transformed, test_size=0.1, random_state=42)

#X_train_imp = newResults.transform(X_train)
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

In [20]:
from sklearn.metrics import accuracy_score
# predictin on the test set
y_pred = clf.predict(X_test)
# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.33629686896018557


In [25]:
# check Important features

feature_importances_df = pd.DataFrame({"feature": list(X.columns), "importance": clf.feature_importances_}
).sort_values("importance", ascending=False)

# Display
feature_importances_df

,feature,importance
3,raceId_x,0.541773
0,driverId,0.271297
1,fastestLap,0.114164
2,rank,0.072767
